# [Predicting the price of Bitcoin with multivariate Pytorch LSTMs - revise](https://charlieoneill.medium.com/predicting-the-price-of-bitcoin-with-multivariate-pytorch-lstms-695bc294130)

In [36]:
'''Train with PyTorch.'''
# PyTorch
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data.dataset import random_split
import torch.utils.data as data

# SciKit
from sklearn.preprocessing import MinMaxScaler, StandardScaler

# Python
import pandas as pd
import numpy as np
import os
import time

In [37]:
df = pd.read_csv('./station/466920taipei_train.csv')
df = df[["觀測時間(hour)", "氣溫(℃)", "相對溼度(%)", "露點溫度(℃)", "測站氣壓(hPa)"]]
df.head()

,觀測時間(hour),氣溫(℃),相對溼度(%),露點溫度(℃),測站氣壓(hPa)
0,2010/1/1 00:00,13.7,69.0,8.2,1021.8
1,2010/1/1 01:00,13.8,69.0,8.3,1021.4
2,2010/1/1 02:00,13.9,69.0,8.4,1020.4
3,2010/1/1 03:00,13.9,69.0,8.4,1019.9
4,2010/1/1 04:00,14.0,69.0,8.5,1019.6


In [38]:
features = df[["相對溼度(%)", "露點溫度(℃)", "測站氣壓(hPa)"]]
targets = df[["氣溫(℃)"]].values

features.shape, targets.shape

((96360, 3), (96360, 1))

In [39]:
mm = MinMaxScaler()
ss = StandardScaler()

features_trans = ss.fit_transform(features)
targets_trans = mm.fit_transform(targets.reshape(-1, 1))

print(targets_trans)
print(targets_trans.shape)

[[0.27298851]
 [0.27586207]
 [0.27873563]
 ...
 [0.1954023 ]
 [0.20402299]
 [0.20402299]]
(96360, 1)


In [40]:
print(type(targets_trans))
print(type(features), type(targets))
print(features_trans[0:100].shape)

<class 'numpy.ndarray'>
<class 'pandas.core.frame.DataFrame'> <class 'numpy.ndarray'>
(100, 3)


In [41]:
def split_sequences(input_sequences, output_sequence, n_steps_in, n_steps_out):
    
    features, targets = list(), list()
    
    for i in range(len(input_sequences)):
        
        # find the end of the input, output sequence
        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out - 1

        # check if we are beyond the dataset
        if out_end_ix > len(input_sequences): break

        # gather input and output of the pattern
        # seq_x: features_trans[0~99](100個)
        # seq_y: targets_trans[99~148:-1](50個)(-1: 2D(3D?)的最後一列)
        seq_x, seq_y = input_sequences[i:end_ix], output_sequence[end_ix-1:out_end_ix, -1]  # 有-1才會是(96212,50)，沒有-1是(96212,50,1)
        features.append(seq_x), targets.append(seq_y)

    return np.array(features), np.array(targets)

features_ss, targets_mm = split_sequences(features_trans, targets_trans, 100, 50)  # feed in 100 samples, up to the current day, and predict the next 50 time step values. 
print(features_ss.shape, targets_mm.shape)

(96212, 100, 3) (96212, 50)


In [44]:
total_samples = len(features)
train_test_split = round(0.7 * total_samples)

features_train = features_ss[:-150]
features_test = features_ss[-150:]

targets_train = targets_mm[:-150]
targets_test = targets_mm[-150:]

print("Training Shape:", features_train.shape, targets_train.shape)
print("Testing Shape:", features_test.shape, targets_test.shape)

Training Shape: (96062, 100, 3) (96062, 50)
Testing Shape: (150, 100, 3) (150, 50)


In [45]:
from torch.autograd import Variable

# convert to pytorch tensors
features_train_tensors = Variable(torch.Tensor(features_train))
features_test_tensors = Variable(torch.Tensor(features_test))

targets_train_tensors = Variable(torch.Tensor(targets_train))
targets_test_tensors = Variable(torch.Tensor(targets_test))

In [46]:
# reshaping to rows, timestamps, features
# batch_size, sequence_length, input size (the number of features)
features_train_tensors_final = torch.reshape(features_train_tensors, 
                                            (features_train_tensors.shape[0], 100, features_train_tensors.shape[2]))
features_test_tensors_final = torch.reshape(features_test_tensors,
                                            (features_test_tensors.shape[0], 100, features_test_tensors.shape[2])) 

print("Training Shape:", features_train_tensors_final.shape, targets_train_tensors.shape)
print("Testing Shape:", features_test_tensors_final.shape, targets_test_tensors.shape) 

Training Shape: torch.Size([96062, 100, 3]) torch.Size([96062, 50])
Testing Shape: torch.Size([150, 100, 3]) torch.Size([150, 50])


In [47]:
features_check, targets_check = split_sequences(features, targets.reshape(-1, 1), 100, 50)

## LSTM model

In [64]:
class LSTM(nn.Module):
    
    def __init__(self, num_classes, input_size, hidden_size, num_layers):
        super().__init__()
        self.num_classes = num_classes # output size
        self.num_layers = num_layers # number of recurrent layers in the lstm
        self.input_size = input_size # input size
        self.hidden_size = hidden_size # neurons in each lstm layer
        # LSTM model
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                            num_layers=num_layers, batch_first=True, dropout=0.2) # lstm
        # self.fc_1 =  nn.Linear(hidden_size, 128) # fully connected 
        # self.fc_2 = nn.Linear(128, num_classes) # fully connected last layer
        self.fc = nn.Linear(hidden_size, num_classes)
        self.relu = nn.ReLU()
        
    def forward(self,x):
        # hidden state
        h_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size))
        # cell state
        c_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size))
        # propagate input through LSTM
        output, (hn, cn) = self.lstm(x, (h_0, c_0)) # (input, hidden, and internal state)
        hn = hn.view(-1, self.hidden_size) # reshaping the data for Dense layer next
        out = self.relu(hn)
        out = self.fc(out)
        # out = self.fc_1(out) # first dense
        # out = self.relu(out) # relu
        # out = self.fc_2(out) # final output
        return out

In [65]:
import warnings
warnings.filterwarnings('ignore')

n_epochs = 100
learning_rate = 0.001

input_size = 3 # number of features
hidden_size = 10 # number of features in hidden state
num_layers = 1 # number of stacked lstm layers

num_classes = 50 # number of output classes 

lstm = LSTM(num_classes, 
              input_size, 
              hidden_size, 
              num_layers)

loss_fn = torch.nn.MSELoss()    # mean-squared error for regression
optimiser = torch.optim.Adam(lstm.parameters(), lr=learning_rate)

In [66]:
lstm.eval()
with torch.no_grad():
    sample_output = lstm(features_train_tensors_final[:1])  # Use a sample input
print("Shape of the output:", sample_output.shape) 

Shape of the output: torch.Size([1, 50])


## Training

In [67]:
def training_loop(n_epochs, lstm, optimiser, loss_fn, X_train, y_train, X_test, y_test):
    for epoch in range(n_epochs):
        lstm.train()
        outputs = lstm.forward(X_train) # forward pass
        optimiser.zero_grad() # calculate the gradient, manually setting to 0
        # obtain the loss function
        loss = loss_fn(outputs, y_train)
        loss.backward() # calculates the loss of the loss function
        optimiser.step() # improve from loss, i.e backprop
        # test loss
        lstm.eval()
        test_preds = lstm(X_test)
        test_loss = loss_fn(test_preds, y_test)
        if epoch % 100 == 0:
            print("Epoch: %d, train loss: %1.5f, test loss: %1.5f" % (epoch, loss.item(), test_loss.item()))

In [68]:
training_loop(n_epochs=n_epochs,
              lstm=lstm,
              optimiser=optimiser,
              loss_fn=loss_fn,
              X_train=features_train_tensors_final,
              y_train=targets_train_tensors,
              X_test=features_test_tensors_final,
              y_test=targets_test_tensors)

Epoch: 0, train loss: nan, test loss: nan


KeyboardInterrupt: 